## Pore size and flow rate distributions in 2D porous media calculator

In [ ]:
import os
import numpy as np
import pyvista as pv
from scipy.ndimage import distance_transform_edt
from scipy.spatial import KDTree
from skimage.morphology import medial_axis
from shapely import vectorized
from shapely.geometry import Point, LineString, box
from shapely.ops import polygonize, unary_union
from shapely.prepared import prep
import networkx as netx
import matplotlib.pyplot as plt

In [ ]:
# We read and load the VTK file that contains the solved OpenFOAM case

case_dir = os.path.expanduser("~/OpenFOAM/jose-v2406/run/YDRAY-flow_n13_sat")
vtk_file = os.path.join(case_dir, "VTK", "YDRAY-flow_n13_sat_1047.vtm")

mb        = pv.read(vtk_file)
vol_mesh  = mb["internal"] #the "fluid" mesh (where CFD is solved)
cyl_patch = mb["boundary"]["wallFluidSolid"] # the walls of the cylindrical obstacles

_, _, _, _, zmin, zmax = vol_mesh.bounds
z_mid = 0.5 * (zmin + zmax)

# As the problem is 2D, we take a 2D slice centered at z=0 of both the internal mesh and the cylinders' walls and get rid of everything else
obst_section = (
    cyl_patch.extract_surface()
             .slice(normal="z", origin=(0, 0, z_mid))
             .clean()
)
dom_section = (
    vol_mesh.extract_surface()
            .slice(normal="z", origin=(0, 0, z_mid))
            .clean()
)

xmin, xmax, ymin, ymax, _, _ = dom_section.bounds



In [ ]:
# (Tu código va aquí arriba)
# ... (importaciones y carga de datos)
# ... (definición de xmin, xmax, ymin, ymax)

# Importamos la función para encontrar picos
from skimage.feature import peak_local_max

print("Iniciando análisis de geometría...")

## 1. Poligonizar los obstáculos
# -------------------------------------------------
# Extraemos los puntos 2D y las líneas de la sección de obstáculos
points_2d = obst_section.points[:, :2]
lines_array = obst_section.lines

shapely_lines = []
i = 0
while i < len(lines_array):
    n_points = lines_array[i]
    point_indices = lines_array[i + 1 : i + 1 + n_points]
    line_coords = points_2d[point_indices]
    
    # Asegurarse de que la línea tenga al menos 2 puntos
    if len(line_coords) > 1:
        shapely_lines.append(LineString(line_coords))
    
    i += n_points + 1

# Usamos polygonize para encontrar todos los polígonos cerrados
all_polygons = list(polygonize(shapely_lines))
print(f"Encontrados {len(all_polygons)} polígonos iniciales.")

## 2. Clasificar polígonos (circulares vs. fusionados)
# -------------------------------------------------
# Un círculo perfecto tiene una circularidad de 1.
# Circularidad = (4 * pi * Area) / (Perimetro^2)
CIRCULARITY_THRESHOLD = 0.9 

circular_polygons = []
merged_polygons = []

for poly in all_polygons:
    if poly.area == 0:
        continue
    
    circularity = (4 * np.pi * poly.area) / (poly.length ** 2)
    
    if circularity > CIRCULARITY_THRESHOLD:
        circular_polygons.append(poly)
    else:
        merged_polygons.append(poly)

print(f"Clasificados: {len(circular_polygons)} círculos individuales y {len(merged_polygons)} polígonos fusionados.")


## 3. Descomponer polígonos fusionados
# -------------------------------------------------
# Esta es la parte más compleja. Usamos una transformada de distancia.

# Definimos una cuadrícula para rasterizar los polígonos
GRID_RES = 500 # Resolución (más alta = más preciso pero más lento)
x_grid = np.linspace(xmin, xmax, GRID_RES)
y_grid = np.linspace(ymin, ymax, GRID_RES)
xx, yy = np.meshgrid(x_grid, y_grid)

# Puntos de la cuadrícula aplanados para la comprobación
grid_points_x = xx.ravel()
grid_points_y = yy.ravel()

# Calculamos el tamaño del píxel (promedio, asumiendo proporciones similares)
pixel_size = (x_grid[1] - x_grid[0] + y_grid[1] - y_grid[0]) / 2.0

# Estimamos una distancia mínima entre picos
# Si encontramos círculos individuales, usamos su radio como guía
if circular_polygons:
    radii = [np.sqrt(p.area / np.pi) for p in circular_polygons]
    median_radius = np.median(radii)
    # Distancia mínima = 80% del radio mediano, convertido a píxeles
    min_dist_px = int((median_radius * 0.8) / pixel_size)
    min_dist_px = max(1, min_dist_px) # Asegura que sea al menos 1
else:
    # Si no hay círculos, usamos un valor fijo (ajustar si es necesario)
    min_dist_px = 5 
    print("Advertencia: No se encontraron círculos individuales. Usando min_distance=5px para la detección de picos.")

print(f"Usando una distancia mínima entre picos de {min_dist_px} píxeles.")

# Lista final para guardar todos los círculos (los originales + los descompuestos)
final_circular_polygons = list(circular_polygons)

for i, merged_poly in enumerate(merged_polygons):
    print(f"Procesando polígono fusionado {i+1}/{len(merged_polygons)}...")
    
    # 1. Rasterizar: crear una máscara 2D del polígono
    mask_flat = vectorized.contains(merged_poly, grid_points_x, grid_points_y)
    mask = mask_flat.reshape(GRID_RES, GRID_RES)
    
    if not np.any(mask):
        print(f"  Polígono {i+1} está vacío o fuera de los límites, omitiendo.")
        continue

    # 2. Transformada de distancia: distancia desde cada píxel interior al borde
    distance = distance_transform_edt(mask)
    
    # 3. Encontrar picos (centros de los círculos)
    # peak_local_max encuentra coordenadas (fila, columna) de los máximos locales
    local_max_indices = peak_local_max(distance, min_distance=min_dist_px, labels=mask)
    
    # 4. Re-crear círculos
    for (row, col) in local_max_indices:
        # Convertir índice de píxel (fila, col) a coordenadas (x, y)
        center_x = x_grid[col]
        center_y = y_grid[row]
        
        # El radio es el valor de la transformada de distancia en el pico
        radius_px = distance[row, col]
        radius = radius_px * pixel_size
        
        # Crear el nuevo círculo y añadirlo a la lista final
        new_circle = Point(center_x, center_y).buffer(radius)
        final_circular_polygons.append(new_circle)

print(f"Descomposición completa. Total de círculos: {len(final_circular_polygons)}")


## 4. Visualización
# -------------------------------------------------
print("Generando visualización...")

fig, ax = plt.subplots(figsize=(12, 12), dpi = 300)

# Dibujar los polígonos originales fusionados (para comparar)
for poly in merged_polygons:
    x, y = poly.exterior.xy
    ax.plot(x, y, color='red', linestyle='--', linewidth=1.5, label='Original Fusionado' if 'Original Fusionado' not in ax.get_legend_handles_labels()[1] else "")

# Dibujar la lista final de círculos
for poly in final_circular_polygons:
    x, y = poly.exterior.xy
    ax.fill(x, y, alpha=0.6, fc='blue', ec='none', label='Círculo Descompuesto' if 'Círculo Descompuesto' not in ax.get_legend_handles_labels()[1] else "")

ax.set_aspect('equal')
ax.set_xlim(xmin, xmax)
ax.set_ylim(ymin, ymax)
ax.set_title(f"Descomposición de Obstáculos ({len(final_circular_polygons)} círculos finales)")
ax.legend()
plt.grid(True, linestyle=':', alpha=0.6)
plt.show()

In [ ]:
print("Definiendo la Región de Interés (ROI)...")

# Calcular la longitud total en X del dominio
x_length = xmax - xmin

# Definir los nuevos límites del ROI (ignorando el 3% en cada extremo)
roi_xmin = xmin + 0.03 * x_length
roi_xmax = xmax - 0.03 * x_length

# Los límites de Y no cambian
roi_ymin = ymin
roi_ymax = ymax

print(f"Límites originales en X: ({xmin:.4f}, {xmax:.4f})")
print(f"Límites del ROI en X:   ({roi_xmin:.4f}, {roi_xmax:.4f})")

In [ ]:
import pandas as pd
from scipy.spatial import Delaunay
from scipy.spatial import cKDTree

print("Iniciando la construcción de la red de tubos...")

## 1. Extraer Centros y Radios
# -------------------------------------------------
# (Esta parte no cambia)
bead_data = []
for poly in final_circular_polygons:
    bead_data.append({
        'center': np.array([poly.centroid.x, poly.centroid.y]),
        'radius': np.sqrt(poly.area / np.pi)
    })
centers = np.array([d['center'] for d in bead_data])
radii = np.array([d['radius'] for d in bead_data])
n_real_beads = len(centers)
print(f"Extraídos {n_real_beads} centros y radios de los granos.")

## 2. Crear "Ghost Beads" para las paredes Ymin / Ymax
# -------------------------------------------------
# (Esta parte no cambia)
max_r = np.max(radii)
reflection_dist = 3.0 * max_r
near_bottom_indices = np.where(centers[:, 1] < ymin + reflection_dist)[0]
ghost_centers_bottom = np.copy(centers[near_bottom_indices])
ghost_centers_bottom[:, 1] = ymin - (ghost_centers_bottom[:, 1] - ymin) # Reflejar
ghost_radii_bottom = radii[near_bottom_indices]
near_top_indices = np.where(centers[:, 1] > ymax - reflection_dist)[0]
ghost_centers_top = np.copy(centers[near_top_indices])
ghost_centers_top[:, 1] = ymax + (ymax - ghost_centers_top[:, 1]) # Reflejar
ghost_radii_top = radii[near_top_indices]
all_centers = np.vstack((centers, ghost_centers_bottom, ghost_centers_top))
all_radii = np.hstack((radii, ghost_radii_bottom, ghost_radii_top))
print(f"Creados {len(ghost_centers_bottom)} ghosts inferiores y {len(ghost_centers_top)} ghosts superiores.")

## 3. Triangulación de Delaunay
# -------------------------------------------------
# (Esta parte no cambia)
tri = Delaunay(all_centers)
edges = set()
for simplex in tri.simplices:
    edges.add(tuple(sorted((simplex[0], simplex[1]))))
    edges.add(tuple(sorted((simplex[1], simplex[2]))))
    edges.add(tuple(sorted((simplex[2], simplex[0]))))
print(f"Triangulación de Delaunay completada. {len(edges)} ejes totales encontrados.")

## 4. Preparar el KDTree para el muestreo rápido
# -------------------------------------------------
# (Esta parte no cambia)
print("Preparando el árbol KDTree para un muestreo de velocidad rápido...")
VEL_NAME_CELL = None
common_names = ['U', 'velocity', 'Velocity', 'VELOCITY']
for name in common_names:
    if name in vol_mesh.cell_data:
        VEL_NAME_CELL = name
        break
if VEL_NAME_CELL is None:
    print(f"ADVERTENCIA: No se encontró 'U' o 'velocity' en vol_mesh.cell_data.")
    print("Buscando primer campo vectorial disponible en cell_data...")
    for name, array in vol_mesh.cell_data.items():
        if array.ndim == 2 and array.shape[1] == 3:
            VEL_NAME_CELL = name
            break
if VEL_NAME_CELL:
    print(f"Usando el campo de velocidad de celda: '{VEL_NAME_CELL}'")
    cell_centers_3d = vol_mesh.cell_centers().points
    U_cells = vol_mesh.cell_data[VEL_NAME_CELL][:, :2] # (M,2)
    tree = cKDTree(cell_centers_3d[:, :2])
    print("Árbol KDTree construido.")
else:
    print("ERROR FATAL: No se encontró ningún campo de velocidad en cell_data. El caudal será 0.")
    tree = None


## 5. Calcular Propiedades del Tubo (FILTRADO POR ROI y SIN DUPLICADOS)
# -------------------------------------------------

H_METERS = 0.001 # 1 mm en metros
N_SAMPLES = 50   # Número de puntos para la integración
tube_list = []

if 'tree' not in locals() or tree is None:
     print("ERROR: El árbol KDTree no está definido. Ejecuta la celda de preparación del árbol.")
if 'roi_xmin' not in locals():
    print("ERROR: Los límites del ROI no están definidos. Ejecuta la celda de definición de ROI.")

wall_tubes_created_for_bead = set()
print(f"Iniciando cálculo de tubos (filtrando por ROI en X: [{roi_xmin:.4f}, {roi_xmax:.4f}])...")

# --- ¡NUEVO! Umbral para definir "flujo real" ---
# Lo usaremos para encontrar la anchura efectiva
VEL_THRESHOLD_RATIO = 0.001 # 5% de la velocidad máxima del perfil

for idx1, idx2 in edges:
    
    # 5.1. Omitir ejes "fantasma-fantasma"
    if idx1 >= n_real_beads and idx2 >= n_real_beads:
        continue

    # 5.2. Geometría del tubo
    is_wall_tube = (idx1 >= n_real_beads) or (idx2 >= n_real_beads)
    
    if is_wall_tube:
        real_idx = idx1 if idx1 < n_real_beads else idx2
        if real_idx in wall_tubes_created_for_bead:
            continue
        c_real = centers[real_idx]
        r_real = radii[real_idx]
        is_top_wall = c_real[1] > (ymax + ymin) / 2.0
        
        if is_top_wall:
            p_edge_bead = np.array([c_real[0], c_real[1] + r_real])
            p_edge_wall = np.array([c_real[0], ymax])
            n_vec = np.array([0.0, 1.0])
        else:
            p_edge_bead = np.array([c_real[0], c_real[1] - r_real])
            p_edge_wall = np.array([c_real[0], ymin])
            n_vec = np.array([0.0, -1.0])
            
        width = np.linalg.norm(p_edge_bead - p_edge_wall)
        wall_tubes_created_for_bead.add(real_idx)
        
    else:
        c1, c2 = all_centers[idx1], all_centers[idx2]
        r1, r2 = all_radii[idx1], all_radii[idx2]
        center_vec = c2 - c1
        dist = np.linalg.norm(center_vec)
        width = dist - (r1 + r2)
        n_vec = center_vec / dist
        p_edge_wall = c1 + n_vec * r1
        p_edge_bead = c2 - n_vec * r2

    # 5.3. Filtrar tubos con solapamiento
    if width <= 0:
        continue

    # --- INICIO DE LA SECCIÓN MODIFICADA ---
    
    # 5.4. Calcular Caudal (Flow Rate) y Anchura Efectiva
    n_vec = np.array([-n_vec[1], n_vec[0]]) # Vector normal para la integral
    flow_rate = 0.0
    effective_width = width # Por defecto, es la anchura geométrica
    
    if tree is not None and width > 1e-9: 
        # Puntos de muestreo a lo largo de la anchura GEOMÉTRICA
        sample_points_2d = np.linspace(p_edge_wall, p_edge_bead, N_SAMPLES)
        # Eje de integración (distancia de 0 a width)
        integration_axis = np.linspace(0, width, N_SAMPLES)
        
        # Muestrear velocidades
        dists, idxs = tree.query(sample_points_2d, k=1)
        velocities_2d = U_cells[idxs]
        velocities_2d = np.nan_to_num(velocities_2d, nan=0.0, posinf=0.0, neginf=0.0)
        
        # Perfil de velocidad normal al tubo
        v_normal_components = np.dot(velocities_2d, n_vec)
        v_abs = np.abs(v_normal_components)
        v_max = np.max(v_abs)
        
        if v_max > 1e-12: # Si hay algo de flujo
            threshold = v_max * VEL_THRESHOLD_RATIO
            
            # Encontrar los índices (del array de N_SAMPLES) donde el flujo es "real"
            effective_indices = np.where(v_abs > threshold)[0]
            
            if len(effective_indices) > 1:
                # Encontrar el inicio y el fin del flujo
                i_start = np.min(effective_indices)
                i_end = np.max(effective_indices)
                
                # Recalcular el caudal (integral) SOLO en la sección efectiva
                v_effective = v_normal_components[i_start:i_end+1]
                axis_effective = integration_axis[i_start:i_end+1]
                
                integral_value = np.trapz(y=v_effective, x=axis_effective)
                flow_rate = np.abs(integral_value) * H_METERS
                
                # ¡LA CLAVE! Corregir la anchura del tubo
                effective_width = axis_effective[-1] - axis_effective[0]
                
            else:
                # Todo el flujo está por debajo del umbral, o es un solo punto
                flow_rate = 0.0
                effective_width = 0.0
                
        else:
            # No hay nada de flujo
            flow_rate = 0.0
            effective_width = 0.0 # Tubo bloqueado
            
    # --- FIN DE LA SECCIÓN MODIFICADA ---

    # 5.5. Calcular Midpoint
    midpoint_2d = (p_edge_wall + p_edge_bead) / 2.0
    
    # 5.6. Guardar resultados (USANDO LA NUEVA ANCHURA)
    tube_list.append({
        'midpoint': midpoint_2d, 
        'width': effective_width, # <-- ¡VALOR CORREGIDO!
        'flow_rate': flow_rate,   # <-- VALOR CORREGIDO!
        'is_wall_tube': is_wall_tube, 
        'bead_indices': (idx1, idx2)
    })

print(f"\n--- Proceso completado ---")
print(f"Se encontraron {len(tube_list)} tubos válidos (width > 0, sin duplicados y dentro del ROI).")

df_tubos = pd.DataFrame(tube_list)
print("\nPrimeros 5 tubos encontrados:")
print(df_tubos.head())

In [ ]:
print("Generando histogramas de distribución (con rango limitado por percentiles)...")

# 1. Preparar los datos
flow_rates = df_tubos['flow_rate']
half_widths = df_tubos['width'] * 0.5

# 2. Calcular los límites del eje X basados en percentiles
#    Por ejemplo, tomamos hasta el percentil 99.5 para excluir los valores más extremos.
#    Puedes ajustar el percentil (e.g., 99, 99.9) si aún ves el problema.
flow_rate_max_limit = np.percentile(flow_rates, 99.5)
half_width_max_limit = np.percentile(half_widths, 99.5)

# Asegurarse de que el límite mínimo sea 0 o muy cerca de 0
flow_rate_min_limit = np.percentile(flow_rates, 0.5) if np.percentile(flow_rates, 0.5) > 0 else 0
half_width_min_limit = np.percentile(half_widths, 0.5) if np.percentile(half_widths, 0.5) > 0 else 0

# 3. Crear la figura con dos subplots
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6), dpi=100)

# --- Histograma de Caudales (Flow Rates) ---
# Definimos los bins dentro del rango limitado
bins_flow_rates = np.linspace(flow_rate_min_limit, flow_rate_max_limit, 20) # Más bins para más detalle
ax1.hist(flow_rates, bins=bins_flow_rates, color='royalblue', alpha=0.75, edgecolor='black')
ax1.set_title(f'Distribución de Caudales (P<={flow_rate_max_limit:.2e})')
ax1.set_xlabel('Caudal (m³/s)')
ax1.set_ylabel('Frecuencia (Conteo)')
#ax1.set_yscale('log')
ax1.grid(True, linestyle=':', alpha=0.6)
# Opcional: Establecer explícitamente los límites del eje X si np.linspace no lo hace perfecto
# ax1.set_xlim(flow_rate_min_limit, flow_rate_max_limit)


# --- Histograma de Mitad de Anchuras (Half-Widths) ---
# Definimos los bins dentro del rango limitado
bins_half_widths = np.linspace(half_width_min_limit, half_width_max_limit, 20) # Más bins para más detalle
ax2.hist(half_widths, bins=bins_half_widths, color='forestgreen', alpha=0.75, edgecolor='black')
ax2.set_title(f'Distribución de Mitad de Anchuras (P<={half_width_max_limit:.2e})')
ax2.set_xlabel('Mitad de Anchura (m)')
ax2.set_ylabel('Frecuencia (Conteo)')
#ax2.set_yscale('log')
ax2.grid(True, linestyle=':', alpha=0.6)
# Opcional: Establecer explícitamente los límites del eje X
# ax2.set_xlim(half_width_min_limit, half_width_max_limit)


# 4. Mostrar el gráfico
plt.tight_layout()
plt.show()

In [ ]:
print("Generando visualización de la Triangulación de Delaunay (Solo Ejes de Imagen)...")

# --- NUEVO: Crear mapa de Fantasma -> Padre Real ---
# Necesitamos 'near_bottom_indices' y 'near_top_indices' de la celda anterior
if 'near_bottom_indices' not in locals():
    print("ERROR: Faltan datos de los granos fantasma. Ejecuta la celda de cálculo de tubos primero.")
else:
    # Esta lista mapea el índice fantasma (relativo, empezando en 0) a su índice padre real
    ghost_parent_index_map = np.hstack((near_bottom_indices, near_top_indices))
    
    # {indice_global_fantasma: indice_global_real_padre}
    ghost_to_parent_dict = {}
    for k, parent_real_idx in enumerate(ghost_parent_index_map):
        ghost_global_idx = n_real_beads + k
        ghost_to_parent_dict[ghost_global_idx] = parent_real_idx
# ----------------------------------------------------


fig, ax = plt.subplots(figsize=(12, 12), dpi=150)

# 1. Dibujar los granos (círculos)
for poly in final_circular_polygons:
    # Optimizacion: no dibujar polígonos que estén completamente fuera del ROI
    if poly.bounds[2] < roi_xmin or poly.bounds[0] > roi_xmax:
        continue
    x, y = poly.exterior.xy
    ax.fill(x, y, alpha=0.3, fc='gray', ec='none', label='Granos' if 'Granos' not in ax.get_legend_handles_labels()[1] else "")

# 2. Dibujar los centros (reales y fantasmas)
# Filtramos centros que caen fuera del ROI en X
real_centers_in_roi = centers[(centers[:, 0] >= roi_xmin) & (centers[:, 0] <= roi_xmax)]
ax.scatter(real_centers_in_roi[:, 0], real_centers_in_roi[:, 1], c='blue', s=5, label='Centros Reales', zorder=5)

ghost_centers = all_centers[n_real_beads:]
ghost_centers_in_roi = ghost_centers[(ghost_centers[:, 0] >= roi_xmin) & (ghost_centers[:, 0] <= roi_xmax)]
ax.scatter(ghost_centers_in_roi[:, 0], ghost_centers_in_roi[:, 1], 
           c='black', marker='x', s=15, label='Centros Fantasma', zorder=5)

# 3. Dibujar los ejes de la triangulación (CON FILTRO)
for idx1, idx2 in edges:
    p1 = all_centers[idx1]
    p2 = all_centers[idx2]
    
    # Optimización: no dibujar ejes que estén completamente fuera del ROI
    if (p1[0] < roi_xmin and p2[0] < roi_xmin) or \
       (p1[0] > roi_xmax and p2[0] > roi_xmax):
        continue

    is_idx1_real = (idx1 < n_real_beads)
    is_idx2_real = (idx2 < n_real_beads)
    
    if is_idx1_real and is_idx2_real:
        # Eje real-real
        ax.plot([p1[0], p2[0]], [p1[1], p2[1]], 
                color='gray', linestyle='--', linewidth=0.7, alpha=0.5,
                label='Eje Real-Real' if 'Eje Real-Real' not in ax.get_legend_handles_labels()[1] else "")
                
    elif is_idx1_real and not is_idx2_real:
        # Eje real-fantasma
        real_idx, ghost_idx = idx1, idx2
        if ghost_idx in ghost_to_parent_dict and real_idx == ghost_to_parent_dict[ghost_idx]:
            ax.plot([p1[0], p2[0]], [p1[1], p2[1]], 
                    color='red', linestyle='-', linewidth=1.0, alpha=0.8,
                    label='Eje de Imagen (Real-Fantasma)' if 'Eje de Imagen (Real-Fantasma)' not in ax.get_legend_handles_labels()[1] else "")
            
    elif not is_idx1_real and is_idx2_real:
        # Eje fantasma-real
        real_idx, ghost_idx = idx2, idx1
        if ghost_idx in ghost_to_parent_dict and real_idx == ghost_to_parent_dict[ghost_idx]:
            ax.plot([p1[0], p2[0]], [p1[1], p2[1]], 
                    color='red', linestyle='-', linewidth=1.0, alpha=0.8,
                    label='Eje de Imagen (Real-Fantasma)' if 'Eje de Imagen (Real-Fantasma)' not in ax.get_legend_handles_labels()[1] else "")

# 4. Configuración del gráfico (CON LÍMITES MODIFICADOS)
ax.set_aspect('equal')
# --- LÍMITE X MODIFICADO ---
ax.set_xlim(roi_xmin, roi_xmax) 
# --- LÍMITE Y SIN CAMBIOS (extendido para fantasmas) ---
ax.set_ylim(ymin - reflection_dist * 0.5, ymax + reflection_dist * 0.5) 

ax.set_title(f"Triangulación de Delaunay (ROI en X de {roi_xmin:.3f} a {roi_xmax:.3f})")
ax.set_xlabel("Coordenada X (m)")
ax.set_ylabel("Coordenada Y (m)")
ax.legend()
plt.grid(True, linestyle=':', alpha=0.6)
plt.show()

In [ ]:
from collections import Counter

print("Construyendo lista de junctions (Método Híbrido: Bulk + Pared)...")

# --- 0. Preparación ---

# Asumimos que estas variables existen de celdas anteriores:
# df_tubos_full: El DF de tubos SIN FILTRO ROI
# tri: La triangulación de Delaunay original (sobre granos+fantasmas)
# edges: El set de ejes de Delaunay
# centers: Array (N,2) de centros de granos reales
# n_real_beads: Número de granos reales
# near_bottom_indices, near_top_indices: Arrays de índices de granos de borde
# roi_xmin, roi_xmax: Límites del ROI

# Función helper para tu lógica de ROI
def is_in_roi(point):
    return roi_xmin <= point[0] <= roi_xmax

# Crear mapas de búsqueda desde la lista de tubos COMPLETA
edge_to_tube_map_full = {} # Clave: (idx1, idx2), Valor: tube_id
wall_tube_map = {}         # Clave: real_bead_idx, Valor: wall_tube_id

print("Creando mapas de búsqueda de tubos (completos)...")
for tube_id, tube_data in df_tubos.iterrows():
    idx1, idx2 = tube_data['bead_indices']
    edge_key = tuple(sorted((idx1, idx2)))
    
    # Mapa de todos los ejes
    edge_to_tube_map_full[edge_key] = tube_id
    
    # Mapa específico para tubos de pared
    if tube_data['is_wall_tube']:
        real_idx = idx1 if idx1 < n_real_beads else idx2
        wall_tube_map[real_idx] = tube_id

# Set de granos que tocan una pared (granos de "borde")
real_border_bead_indices = set(near_bottom_indices) | set(near_top_indices)

# Esta será nuestra nueva lista de junctions
# Contendrá diccionarios para más estructura
junctions_list_structured = []

# --- 1. Calcular Junctions "BULK" ---
print("Procesando junctions 'Bulk' (interiores)...")

for simplex in tri.simplices:
    idx1, idx2, idx3 = simplex
    
    # Condición: Triángulo formado UNICAMENTE por granos reales
    if not (idx1 < n_real_beads and idx2 < n_real_beads and idx3 < n_real_beads):
        continue
        
    # Aplicar tu lógica de ROI
    c1, c2, c3 = centers[idx1], centers[idx2], centers[idx3]
    roi_count = sum(is_in_roi(c) for c in [c1, c2, c3])
    
    if roi_count == 0:
        continue # Descartar (ningún grano en ROI)
    
    # Si roi_count > 0, guardamos la junction (tu Lógica 1 y 2)
    j_coord = (c1 + c2 + c3) / 3.0
    
    # Encontrar los 3 tubos que la forman
    edge1 = tuple(sorted((idx1, idx2)))
    edge2 = tuple(sorted((idx2, idx3)))
    edge3 = tuple(sorted((idx3, idx1)))
    
    tube_ids = [
        edge_to_tube_map_full.get(edge1),
        edge_to_tube_map_full.get(edge2),
        edge_to_tube_map_full.get(edge3)
    ]
    
    # Guardamos solo los tubos que existen (width > 0)
    valid_tube_ids = [t_id for t_id in tube_ids if t_id is not None]
    
    if valid_tube_ids: # Solo guardar si tiene al menos un tubo válido
        junctions_list_structured.append({
            'coord': j_coord,
            'tubes': valid_tube_ids,
            'type': 'bulk'
        })

print(f"Encontradas {len(junctions_list_structured)} junctions 'Bulk'.")

# --- 2. Calcular Junctions "DE PARED" ---
print("Procesando junctions 'de Pared'...")
wall_junction_count = 0

for idx1, idx2 in edges: # Iterar sobre *todos* los ejes de Delaunay
    
    # Condición: Ambos deben ser granos "de borde"
    is_border1 = idx1 in real_border_bead_indices
    is_border2 = idx2 in real_border_bead_indices
    
    if not (is_border1 and is_border2):
        continue
        
    # Condición: Ambos centros deben estar DENTRO del ROI
    c1, c2 = centers[idx1], centers[idx2]
    if not (is_in_roi(c1) and is_in_roi(c2)):
        continue
        
    # Si todo se cumple, encontrar los 3 tubos
    
    # Tubo 1: El que conecta los dos granos reales
    tube_bulk_id = edge_to_tube_map_full.get(tuple(sorted((idx1, idx2))))
    
    # Tubo 2: El tubo de pared del grano 1
    tube_wall1_id = wall_tube_map.get(idx1)
    
    # Tubo 3: El tubo de pared del grano 2
    tube_wall2_id = wall_tube_map.get(idx2)
    
    # Asegurarse de que los 3 tubos existen (width > 0)
    if tube_bulk_id is None or tube_wall1_id is None or tube_wall2_id is None:
        continue
        
    # Calcular coordenadas de la junction
    midpoint1 = df_tubos.loc[tube_wall1_id]['midpoint']
    midpoint2 = df_tubos.loc[tube_wall2_id]['midpoint']
    j_coord = (midpoint1 + midpoint2) / 2.0
    
    # Guardar la junction
    junctions_list_structured.append({
        'coord': j_coord,
        'tubes': [tube_bulk_id, tube_wall1_id, tube_wall2_id],
        'type': 'wall'
    })
    wall_junction_count += 1

print(f"Encontradas {wall_junction_count} junctions 'de Pared'.")

# --- 3. Resumen Final ---

# Para mantener compatibilidad con tu código anterior,
# creamos la 'junctions_list' solo con las listas de tubos.
junctions_list = [j['tubes'] for j in junctions_list_structured]

print(f"\n--- Proceso completado ---")
print(f"Se han encontrado {len(junctions_list)} junctions totales (Bulk + Pared).")

# Analizar la "coordinación" (cuántos tubos por junction)
junction_sizes = [len(j) for j in junctions_list]
size_counts = Counter(junction_sizes)
type_counts = Counter(j['type'] for j in junctions_list_structured)

print("\nDistribución por Tipo:")
for j_type, count in type_counts.items():
    print(f"  - Junctions de tipo '{j_type}': {count}")

print("\nDistribución de tubos por junction:")
for size, count in sorted(size_counts.items()):
    print(f"  - Junctions con {size} tubos: {count}")

# (Opcional) Mostrar las primeras 5 junctions
print("\nEjemplo (primeras 5 junctions):")
for i, j_data in enumerate(junctions_list_structured[:5]):
    print(f"  Junction {i} (tipo {j_data['type']}): tubos {j_data['tubes']}")

In [ ]:
print("Filtrando la lista de tubos y las junctions por el ROI...")

# 1. Filtrar la lista principal de tubos (df_tubos)
#    (El df_tubos actual es el 'full')
midpoints = np.stack(df_tubos['midpoint'].values)
in_roi_mask = (midpoints[:, 0] >= roi_xmin) & (midpoints[:, 0] <= roi_xmax)

# Este es el nuevo DataFrame filtrado que querías
df_tubos_roi = df_tubos[in_roi_mask].copy()

print(f"Lista de tubos filtrada: {len(df_tubos)} -> {len(df_tubos_roi)} tubos en ROI.")

# 2. Obtener los IDs (índices) de los tubos que SÍ están en el ROI
valid_tube_ids = set(df_tubos_roi.index)

# 3. Filtrar la lista de junctions (junctions_list_structured)
junctions_list_filtered = []
junctions_list_structured_filtered = []

for junction in junctions_list_structured:
    # Filtrar la lista de tubos de esta junction
    filtered_tubes = [
        tube_id for tube_id in junction['tubes'] 
        if tube_id in valid_tube_ids
    ]
    
    # Opcional: No guardar junctions que se queden con < 2 tubos
    # (una junction real debe conectar al menos 2 tubos)
    if len(filtered_tubes) > 1:
        new_junction_data = junction.copy()
        new_junction_data['tubes'] = filtered_tubes
        
        junctions_list_structured_filtered.append(new_junction_data)
        junctions_list_filtered.append(filtered_tubes)

print(f"Lista de junctions filtrada: {len(junctions_list_structured)} -> {len(junctions_list_structured_filtered)} junctions válidas.")

# 4. Reemplazar las variables viejas por las nuevas filtradas
#    A partir de ahora, df_tubos y junctions_list estarán filtrados por el ROI.
df_tubos = df_tubos_roi
junctions_list = junctions_list_filtered
junctions_list_structured = junctions_list_structured_filtered

print("Variables 'df_tubos' y 'junctions_list' actualizadas.")

In [ ]:
from collections import Counter

print("Refinando la topología de la red (Manejando 'loose ends')...")

# Asumimos que 'df_tubos' y 'junctions_list_structured' ya están filtrados por el ROI.

# --- 1. Contar apariciones de cada tubo en las junctions ---
# Contamos cuántas junctions "tocan" cada tubo
tube_appearance_count = Counter()
for junction in junctions_list_structured:
    for tube_id in junction['tubes']:
        tube_appearance_count[tube_id] += 1

# --- 2. Identificar "loose ends" y clasificarlos ---
# (Tubos que aparecen 0 o 1 vez)

new_junctions_to_add = []
tubes_to_delete = set()

# Definimos una tolerancia para los bordes (2% del ancho del ROI)
roi_width = roi_xmax - roi_xmin
tolerance = roi_width * 0.02
left_boundary = roi_xmin + tolerance
right_boundary = roi_xmax - tolerance

print(f"Definida tolerancia de borde en X: {tolerance:.4f} m")

# Iteramos sobre TODOS los tubos en nuestra lista filtrada por ROI
for tube_id in df_tubos.index:
    count = tube_appearance_count.get(tube_id, 0)
    
    # Buscamos tubos con 0 o 1 conexión
    if count == 0 or count == 1:
        
        # Es un "loose end". Creamos una junction candidata.
        midpoint = df_tubos.loc[tube_id]['midpoint']
        j_coord = midpoint
        
        # Comprobar si es un inlet/outlet (cerca del borde)
        is_near_left = (j_coord[0] <= left_boundary)
        is_near_right = (j_coord[0] >= right_boundary)
        
        if is_near_left or is_near_right:
            # Es un inlet/outlet VÁLIDO.
            # Creamos la nueva junction de 1 tubo.
            new_junctions_to_add.append({
                'coord': j_coord,
                'tubes': [tube_id],
                'type': 'outlet' # Nueva categoría
            })
        else:
            # Es un "dangling tube" INVÁLIDO (en medio del bulk).
            # Lo marcamos para borrar.
            tubes_to_delete.add(tube_id)

print(f"Identificados {len(new_junctions_to_add)} tubos de inlet/outlet.")
print(f"Identificados {len(tubes_to_delete)} tubos 'dangling' internos para borrar.")

# --- 3. Filtrar las listas (¡El paso clave!) ---

# 3.1. Eliminar los tubos 'dangling' de la lista principal
df_tubos_final = df_tubos.drop(index=list(tubes_to_delete))

# 3.2. Filtrar la lista de junctions existente
final_junctions_structured = []
for junction in junctions_list_structured:
    
    # Re-crear la lista de tubos de la junction,
    # omitiendo los que marcamos para borrar
    new_tube_list = [
        t_id for t_id in junction['tubes'] 
        if t_id not in tubes_to_delete
    ]
    
    # Solo mantenemos la junction si todavía tiene tubos
    if len(new_tube_list) > 0:
        junction['tubes'] = new_tube_list
        final_junctions_structured.append(junction)

# 3.3. Añadir las nuevas junctions de inlet/outlet
final_junctions_structured.extend(new_junctions_to_add)

# --- 4. Reemplazar las variables globales ---

df_tubos = df_tubos_final
junctions_list_structured = final_junctions_structured
junctions_list = [j['tubes'] for j in junctions_list_structured] # Regenerar la lista simple

print(f"\n--- Limpieza completada ---")
print(f"Lista final de tubos: {len(df_tubos)}")
print(f"Lista final de junctions: {len(junctions_list)}")

# (Opcional) Ver la nueva distribución de tipos
final_type_counts = Counter(j['type'] for j in junctions_list_structured)
print("\nNueva distribución de tipos de Junction:")
for j_type, count in final_type_counts.items():
    print(f"  - Junctions de tipo '{j_type}': {count}")

In [ ]:
from collections import Counter
import numpy as np

print("Iniciando filtro final de 'border-to-border'...")

# --- 1. Identificar Junctions de Borde ---

# (Asumimos que 'junctions_list_structured', 'df_tubos', 'roi_xmin', 'roi_xmax' existen)

# Re-definimos la tolerancia (igual que en la celda anterior)
roi_width = roi_xmax - roi_xmin
tolerance = roi_width * 0.02
left_boundary = roi_xmin + tolerance
right_boundary = roi_xmax - tolerance

# Guardamos los ÍNDICES de la lista 'junctions_list_structured'
border_junction_indices = set()
print(f"Marcando junctions 'de borde' (tolerancia X: {tolerance:.4f} m)...")

for j_index, junction in enumerate(junctions_list_structured):
    coord = junction['coord']
    if coord[0] <= left_boundary or coord[0] >= right_boundary:
        border_junction_indices.add(j_index)

print(f"Se identificaron {len(border_junction_indices)} junctions 'de borde'.")

# --- 2. Crear mapa Inverso (Tubo -> Junctions) ---
# Necesitamos saber qué dos junctions conecta cada tubo

# (Usamos 'junctions_list_structured' y 'df_tubos' actuales)
tube_to_junctions_map = {tube_id: [] for tube_id in df_tubos.index}

for j_index, junction in enumerate(junctions_list_structured):
    for tube_id in junction['tubes']:
        # Asegurarnos de que el tubo existe en el mapa (no debería fallar)
        if tube_id in tube_to_junctions_map:
            tube_to_junctions_map[tube_id].append(j_index)

# --- 3. Identificar Tubos "Border-to-Border" ---
tubes_to_delete_b2b = set()

for tube_id, connected_junction_indices in tube_to_junctions_map.items():
    
    # Solo nos interesan los tubos que conectan EXACTAMENTE 2 junctions
    if len(connected_junction_indices) == 2:
        j_idx_1 = connected_junction_indices[0]
        j_idx_2 = connected_junction_indices[1]
        
        # Si AMBAS junctions son "de borde", marcamos el tubo para borrar
        if j_idx_1 in border_junction_indices and j_idx_2 in border_junction_indices:
            tubes_to_delete_b2b.add(tube_id)

print(f"Se identificaron {len(tubes_to_delete_b2b)} tubos 'border-to-border' para eliminar.")

# --- 4. Aplicar Filtros y Limpiar ---

# 4.1. Eliminar los tubos del DataFrame principal
df_tubos_final_b2b = df_tubos.drop(index=list(tubes_to_delete_b2b))

# 4.2. Filtrar la lista de junctions (eliminando tubos y junctions vacías)
final_junctions_structured_b2b = []
for j_index, junction in enumerate(junctions_list_structured):
    
    # Recrear la lista de tubos de la junction,
    # omitiendo los que acabamos de borrar
    new_tube_list = [
        t_id for t_id in junction['tubes'] 
        if t_id not in tubes_to_delete_b2b
    ]
    
    # Si la junction se ha quedado sin tubos, la eliminamos
    if len(new_tube_list) > 0:
        junction['tubes'] = new_tube_list
        final_junctions_structured_b2b.append(junction)

# --- 5. Reemplazar las variables globales ---
df_tubos = df_tubos_final_b2b
junctions_list_structured = final_junctions_structured_b2b
junctions_list = [j['tubes'] for j in junctions_list_structured] # Regenerar la lista simple

# --- 6. (NUEVO) Re-calcular border_junction_indices ---
# (Porque la lista ha cambiado de tamaño y los índices se han desplazado)
print("Re-calculando el set 'border_junction_indices' final...")
border_junction_indices = set()
for j_index, junction in enumerate(junctions_list_structured):
    coord = junction['coord']
    if coord[0] <= left_boundary or coord[0] >= right_boundary:
        border_junction_indices.add(j_index)

print(f"\n--- Filtro 'border-to-border' completado ---")
print(f"Lista final de tubos: {len(df_tubos)}")
print(f"Lista final de junctions: {len(junctions_list)}")
print(f"Set de junctions de borde actualizado a {len(border_junction_indices)}.")

In [ ]:
import matplotlib.pyplot as plt

print("Generando visualización de la Red de Poros (Junctions y Tubos)...")

# Asumimos que 'final_circular_polygons', 'df_tubos', 
# 'junctions_list_structured', 'roi_xmin', 'roi_xmax', 
# 'ymin', 'ymax' están definidos y filtrados.

# 1. Crear un mapa de búsqueda para los midpoints de los tubos
#    (Usamos el df_tubos ya filtrado por el ROI)
tube_midpoints_map = {idx: row['midpoint'] for idx, row in df_tubos.iterrows()}

# 2. Crear la figura con alta resolución
fig, ax = plt.subplots(figsize=(14, 14), dpi=200) # <-- DPI elevado

# 3. Dibujar los granos (círculos) en el fondo
for poly in final_circular_polygons:
    # Optimización: no dibujar polígonos que estén completamente fuera del ROI
    if poly.bounds[2] < roi_xmin or poly.bounds[0] > roi_xmax:
        continue
    x, y = poly.exterior.xy
    ax.fill(x, y, alpha=0.3, fc='gray', ec='none', label='Granos' if 'Granos' not in ax.get_legend_handles_labels()[1] else "")

# 4. Preparar listas para dibujar las junctions y conexiones
junction_coords = []
connection_lines = [] # Lista de [[x_j, y_j], [x_t, y_t]]

# Iteramos sobre la lista estructurada, que tiene las coordenadas
for junction in junctions_list_structured:
    j_center = junction['coord']
    tube_ids = junction['tubes']
    
    # Guardamos el centro de la junction
    junction_coords.append(j_center)
    
    # Guardamos las líneas de conexión
    for tube_id in tube_ids:
        # Buscamos el midpoint del tubo (debería existir, ya que todo está filtrado)
        midpoint = tube_midpoints_map.get(tube_id)
        if midpoint is not None:
            connection_lines.append([j_center, midpoint])

# 5. Dibujar las conexiones (líneas verdes)
for line in connection_lines:
    p_junction = line[0]
    p_midpoint = line[1]
    ax.plot([p_junction[0], p_midpoint[0]], [p_junction[1], p_midpoint[1]], 
            color='lime', linestyle='-', linewidth=1.0, alpha=0.8, zorder=8,
            label='Conexión Junction-Tubo' if 'Conexión Junction-Tubo' not in ax.get_legend_handles_labels()[1] else "")

# 6. Dibujar los centros de las junctions (puntos morados)
if junction_coords:
    junction_coords_np = np.array(junction_coords)
    ax.scatter(junction_coords_np[:, 0], junction_coords_np[:, 1], 
               color='purple', 
               s=10,  # <-- Puntos pequeños
               marker='o', 
               zorder=10, 
               label='Centro de Junction' if 'Centro de Junction' not in ax.get_legend_handles_labels()[1] else "")

# 7. Configuración del gráfico
ax.set_aspect('equal')
ax.set_xlim(xmin, xmax)
ax.set_ylim(ymin, ymax) # <-- Usamos los límites físicos (sin fantasmas)
ax.set_title(f"Red de Poros Final (Junctions y Tubos en ROI)")
ax.set_xlabel("Coordenada X (m)")
ax.set_ylabel("Coordenada Y (m)")
ax.legend()
plt.grid(True, linestyle=':', alpha=0.6)
plt.show()

print(f"Visualización completada. {len(junction_coords)} junctions en el ROI.")

In [ ]:
import numpy as np

print("Pre-calculando direcciones de tubos y flujos In/Out...")

# --- 1. Mapas de búsqueda (si no existen) ---
if 'tube_flow_map' not in locals():
    tube_flow_map = {idx: row['flow_rate'] for idx, row in df_tubos.iterrows()}

# Mapa de tubos -> [junction_idx_A, junction_idx_B]
tube_to_junctions_map = {tube_id: [] for tube_id in df_tubos.index}
for j_index, junction_tubes in enumerate(junctions_list):
    for tube_id in junction_tubes:
        if tube_id in tube_to_junctions_map:
            tube_to_junctions_map[tube_id].append(j_index)

# --- 2. Calcular la dirección de CADA tubo ---
# (Usando la lógica de la Tangente Orientada)
tube_direction_map = {} # {tube_id: (j_initial_id, j_final_id)}
print("Calculando dirección de flujo para todos los tubos...")

for tube_id, tube_data in df_tubos.iterrows():
    
    # 2.1. Encontrar las dos junctions del tubo
    junction_indices = tube_to_junctions_map.get(tube_id, [])
    if len(junction_indices) != 2:
        continue # Omitir tubos sin 2 junctions (no debería pasar)
        
    j_idx_A, j_idx_B = junction_indices
    j_coord_A = junctions_list_structured[j_idx_A]['coord']
    j_coord_B = junctions_list_structured[j_idx_B]['coord']

    # 2.2. Recalcular geometría (para obtener n_vec_path)
    idx1, idx2 = tube_data['bead_indices']
    is_wall_tube = tube_data['is_wall_tube']
    width = tube_data['width']
    
    if is_wall_tube:
        real_idx = idx1 if idx1 < n_real_beads else idx2
        c_real = centers[real_idx]; r_real = radii[real_idx]
        is_top_wall = c_real[1] > (ymax + ymin) / 2.0
        if is_top_wall:
            p_edge_bead = np.array([c_real[0], c_real[1] + r_real])
            p_edge_wall = np.array([c_real[0], ymax])
            n_vec_path = np.array([0.0, 1.0])
        else:
            p_edge_bead = np.array([c_real[0], c_real[1] - r_real])
            p_edge_wall = np.array([c_real[0], ymin])
            n_vec_path = np.array([0.0, -1.0])
    else:
        c1, c2 = all_centers[idx1], all_centers[idx2]
        r1, r2 = all_radii[idx1], all_radii[idx2]
        center_vec = c2 - c1; dist = np.linalg.norm(center_vec)
        n_vec_path = center_vec / dist
        p_edge_wall = c1 + n_vec_path * r1
        p_edge_bead = c2 - n_vec_path * r2
        
    # 2.3. Calcular Tangente Orientada
    n_vec_flow_initial = np.array([-n_vec_path[1], n_vec_path[0]])
    t_oriented = n_vec_flow_initial 

    if tree is not None and width > 1e-9:
        sample_points_2d = np.linspace(p_edge_wall, p_edge_bead, N_SAMPLES)
        dists, idxs = tree.query(sample_points_2d, k=1)
        velocities_2d = U_cells[idxs]
        velocities_2d = np.nan_to_num(velocities_2d, nan=0.0, posinf=0.0, neginf=0.0)
        
        proj_mean = np.dot(velocities_2d, n_vec_flow_initial).mean()
        sign = np.sign(proj_mean) if np.sign(proj_mean) != 0 else 1.0
        t_oriented = n_vec_flow_initial * sign
    
    # 2.4. Determinar junction Inicial y Final
    midpoint = tube_data['midpoint'] 
    vec_m_to_A = j_coord_A - midpoint
    dp_A = t_oriented.dot(vec_m_to_A)
    
    if dp_A > 0:
        j_initial_id, j_final_id = j_idx_B, j_idx_A
    else:
        j_initial_id, j_final_id = j_idx_A, j_idx_B
        
    # Guardar el resultado
    tube_direction_map[tube_id] = (j_initial_id, j_final_id)

print(f"Dirección calculada para {len(tube_direction_map)} tubos.")

# --- 3. Calcular Flujos In/Out para CADA junction ---
print("Calculando flujos In/Out por junction...")

# Inicializar mapas con ceros para todas las junctions
junction_flow_in = {j: 0.0 for j in range(len(junctions_list_structured))}
junction_flow_out = {j: 0.0 for j in range(len(junctions_list_structured))}

# Llenar los mapas
for tube_id, (j_initial, j_final) in tube_direction_map.items():
    flow = tube_flow_map.get(tube_id, 0.0)
    
    # Sumar al Out de la junction inicial
    junction_flow_out[j_initial] += flow
    
    # Sumar al In de la junction final
    junction_flow_in[j_final] += flow

print("Pre-cálculo completado.")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

print("Generando gráficos 'Flow In' vs 'Flow Out' para cada junction...")

# --- Usamos los mapas pre-calculados 'junction_flow_in' y 'junction_flow_out' ---

all_flow_in = []
all_flow_out = []

# Iteramos sobre todas las junctions
for junction_id in range(len(junctions_list_structured)):
    
    # Ignorar junctions de borde (inlets/outlets)
    if junction_id in border_junction_indices:
        continue
        
    all_flow_in.append(junction_flow_in[junction_id])
    all_flow_out.append(junction_flow_out[junction_id])

# 3. Convertir a arrays de NumPy
flow_in_np = np.array(all_flow_in)
flow_out_np = np.array(all_flow_out)

print(f"Cálculo completado para {len(flow_in_np)} junctions internas.")

# --- 4. Generar Gráficos ---

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 7), dpi=100)

# --- Gráfico 1: Scatter Plot (Flow Out vs Flow In) ---

# Calcular un límite visual
if len(flow_in_np) > 0:
    combined_max = np.percentile(np.hstack((flow_in_np, flow_out_np)), 99.5)
    if combined_max == 0: 
        combined_max = np.max(np.hstack((flow_in_np, flow_out_np)))
else:
    combined_max = 1.0 # Valor por defecto

ax1.scatter(flow_out_np, flow_in_np, alpha=0.4, s=10, label='Junctions Internas')
ax1.plot([0, combined_max], [0, combined_max], 'r--', linewidth=2, label='Recta y=x')
ax1.set_xlabel('Flow Out (m³/s)') # <-- ETIQUETA CAMBIADA
ax1.set_ylabel('Flow In (m³/s)')  # <-- ETIQUETA CAMBIADA
ax1.set_title('Conservación de Caudal (Junctions Internas)')
ax1.set_aspect('equal')
ax1.set_xlim(0, combined_max)
ax1.set_ylim(0, combined_max)
ax1.legend()
ax1.grid(True, linestyle=':', alpha=0.6)

# --- Gráfico 2: PDF Normalizadas ---

bins = np.linspace(0, combined_max, 50)

ax2.hist(flow_out_np, bins=bins, density=True, alpha=0.7, color='blue', label='PDF Flow Out') # <-- ETIQUETA CAMBIADA
ax2.hist(flow_in_np, bins=bins, density=True, alpha=0.7, color='red', label='PDF Flow In')   # <-- ETIQUETA CAMBIADA
ax2.set_xlabel('Caudal (m³/s)')
ax2.set_ylabel('Densidad de Probabilidad (Normalizada)')
ax2.set_title('Distribución de Caudales (Junctions Internas)')
ax2.legend()
ax2.grid(True, linestyle=':', alpha=0.6)

plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

print("Identificando y visualizando junctions outliers...")

# --- 1. Re-identificar Outliers (mismo código de antes) ---
#    (Pero esta vez guardamos los datos de la junction)
    
RELATIVE_TOLERANCE = 0.96 
ABSOLUTE_TOLERANCE = 1e-12 

# Asumimos que 'junctions_list_structured' y 'tube_flow_map' existen
outlier_junctions_data = [] # <-- Lista para guardar los datos

for junction in junctions_list_structured:
    
    if junction_id in border_junction_indices:
        continue

    tube_id_list = junction['tubes']
    flows = [tube_flow_map[t_id] for t_id in tube_id_list if t_id in tube_flow_map]
    
    if len(flows) <= 1:
        continue
    
    flow_big = max(flows)
    flow_smalls = sum(flows) - flow_big
    
    # Si es un outlier, guardamos la junction entera (que tiene 'coord' y 'tubes')
    if flow_big > ABSOLUTE_TOLERANCE and flow_smalls < (flow_big * RELATIVE_TOLERANCE):
        outlier_junctions_data.append(junction)

print(f"Se encontraron {len(outlier_junctions_data)} junctions outliers para visualizar.")

# --- 2. Crear la Visualización ---
# (Asumimos que 'final_circular_polygons', 'roi_xmin', 'roi_xmax', 'ymin', 'ymax' existen)

fig, ax = plt.subplots(figsize=(14, 14), dpi=150)

# 2.1. Dibujar los granos (fondo)
for poly in final_circular_polygons:
    # Optimización de dibujado
    if poly.bounds[2] < roi_xmin or poly.bounds[0] > roi_xmax:
        continue
    x, y = poly.exterior.xy
    ax.fill(x, y, alpha=0.3, fc='gray', ec='none', label='Granos' if 'Granos' not in ax.get_legend_handles_labels()[1] else "")

# 2.2. Preparar listas para dibujar las junctions y conexiones OUTLIER
outlier_centers_coords = []
outlier_connection_lines = [] # Lista de [[x_j, y_j], [x_t, y_t]]

for junction in outlier_junctions_data:
    j_center = junction['coord']
    tube_ids = junction['tubes']
    
    outlier_centers_coords.append(j_center)
    
    for tube_id in tube_ids:
        midpoint = tube_midpoints_map.get(tube_id) # Usamos el mapa de la celda anterior
        if midpoint is not None:
            outlier_connection_lines.append([j_center, midpoint])

# 2.3. Dibujar las conexiones (líneas rojas)
for line in outlier_connection_lines:
    p_junction = line[0]
    p_midpoint = line[1]
    ax.plot([p_junction[0], p_midpoint[0]], [p_junction[1], p_midpoint[1]], 
            color='red', linestyle='-', linewidth=2.0, alpha=1.0, zorder=10,
            label='Conexión Outlier' if 'Conexión Outlier' not in ax.get_legend_handles_labels()[1] else "")

# 2.4. Dibujar los centros de las junctions (círculos rojos)
if outlier_centers_coords:
    coords_np = np.array(outlier_centers_coords)
    ax.scatter(coords_np[:, 0], coords_np[:, 1], 
               color='red', 
               s=30,  # <-- Puntos más grandes para verlos bien
               marker='o', 
               edgecolor='black', # Borde negro para destacar
               zorder=12, 
               label='Junction Outlier' if 'Junction Outlier' not in ax.get_legend_handles_labels()[1] else "")

# 2.5. Configuración del gráfico
ax.set_aspect('equal')
ax.set_xlim(roi_xmin, roi_xmax)
ax.set_ylim(ymin, ymax) 
ax.set_title(f"Visualización de Junctions con Desbalance de Caudal ({len(outlier_junctions_data)} outliers)")
ax.set_xlabel("Coordenada X (m)")
ax.set_ylabel("Coordenada Y (m)")
ax.axvline(x=left_boundary, color='cyan', linestyle=':', linewidth=2, label='Límite de Borde')
ax.axvline(x=right_boundary, color='cyan', linestyle=':', linewidth=2)
ax.legend()
plt.grid(True, linestyle=':', alpha=0.6)
plt.show()

In [ ]:
import numpy as np

print("Iniciando la exportación de la red a archivos...")

# --- Asumimos que estas variables existen de las celdas anteriores ---
# df_tubos, junctions_list_structured, junction_coords, output_array
# tube_direction_map (¡NUEVO!)
# junction_flow_out (¡NUEVO!)
# tube_flow_map (¡NUEVO!)
# ---------------------------------------------


# --- 1. Guardar "junction_coordinates.txt" (CON NODE ID) ---
# (Esta parte no cambia)
print("Guardando junction_coordinates.txt...")
if 'output_array' not in locals():
    junction_coords = np.array([j['coord'] for j in junctions_list_structured])
    node_ids = np.arange(len(junction_coords))
    node_ids_col = node_ids.reshape(-1, 1)
    output_array = np.hstack((node_ids_col, junction_coords))

np.savetxt(
    "junction_coordinates.txt", 
    output_array, 
    fmt=['%d', '%.8f', '%.8f'],
    comments=""
)

# --- 2. Preparaciones para "junction_links.txt" ---
# (Esta sección se simplifica enormemente)
print("Preparando mapas para links...")
# Todos los mapas complejos (tube_direction_map, junction_flow_out, etc.)
# ya han sido calculados en la celda anterior.

# --- ¡NUEVO! PRE-CÁLCULO PARA NODOS ESTANCADOS ---
# Identificar nodos con flujo saliente CERO y contar sus salidas topológicas
print("Pre-calculando salidas para nodos estancados...")
stagnation_node_out_counts = {}

# 1. Encontrar todos los nodos estancados (flow_out <= 1e-12)
stagnation_node_ids = set()
for j_index in range(len(junctions_list_structured)):
    # Usamos el umbral para ser consistentes
    if junction_flow_out.get(j_index, 0.0) <= 1e-12:
        stagnation_node_ids.add(j_index)

# 2. Contar las salidas TOPOLÓGICAS (cuántos tubos salen de ellos)
#    (Inicializar contador a 0 para todos los nodos)
node_out_link_count = {j_id: 0 for j_id in range(len(junctions_list_structured))}
for tube_id, (j_init, j_fin) in tube_direction_map.items():
    # Usamos .get() para no fallar si j_init no estuviera (aunque debería)
    if j_init in node_out_link_count:
        node_out_link_count[j_init] += 1

# 3. Guardar el recuento final SOLO para los nodos estancados
for j_id in stagnation_node_ids:
    stagnation_node_out_counts[j_id] = node_out_link_count.get(j_id, 0)

print(f"Encontrados {len(stagnation_node_out_counts)} nodos estancados con salidas topológicas.")
# --- FIN DEL NUEVO BLOQUE ---

# --- 3. Recorrer tubos y generar links ---
print("Generando links...")
links_data = []

# Iteramos sobre el mapa de direcciones que ya calculamos
for tube_id, (j_initial_id, j_final_id) in tube_direction_map.items():
    
    flow_this_tube = tube_flow_map.get(tube_id, 0.0)
    flow_out_initial = junction_flow_out.get(j_initial_id, 0.0)
    
    weight = 0.0
    
    if flow_out_initial > 1e-12:
        # --- Caso 1: Flujo Normal ---
        # El nodo NO está estancado. Usamos la fracción de flujo real.
        weight = flow_this_tube / flow_out_initial
        weight = min(weight, 1.0) # Asegurarse de que no sea > 1
    
    else:
        # --- Caso 2: Nodo Estancado (flow_out_initial es 0) ---
        # j_initial_id es un nodo estancado. Debemos asignar pesos
        # de forma equiprobable.
        N_out = stagnation_node_out_counts.get(j_initial_id, 0)
        
        if N_out > 0:
            # Asignar peso equiprobable 1/N
            weight = 1.0 / N_out
        else:
            # Este nodo no tiene salidas topológicas,
            # el peso es 0 (y este link no debería existir,
            # pero lo manejamos por si acaso)
            weight = 0.0

    # --- Filtro Final ---
    # Guardamos el link si el peso es significativo.
    # Esto incluye los links (peso > 0) del Caso 1
    # Y los links (peso = 1/N > 0) del Caso 2.
    # Solo filtra los links con flow_this_tube = 0 del Caso 1.
    if weight > 1e-9:
        links_data.append([j_initial_id, j_final_id, weight])

# --- 4. Guardar "junction_links.txt" ---
# (Tu código np.savetxt va aquí...)
# --- 4. Guardar "junction_links.txt" ---
print("Guardando junction_links.txt...")
links_array = np.array(links_data)
np.savetxt(
    "junction_links.txt",
    links_array,
    fmt=['%d', '%d', '%.8f'], # [int, int, float]
    comments=""
)

print(f"\n--- Exportación completada ---")
print(f"Se guardaron {len(output_array)} coordenadas.")
print(f"Se guardaron {len(links_array)} links.")

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

print("Generando visualización de la red con flechas de flujo...")

# --- 1. Cargar datos (si no están ya en memoria) ---
# (Normalmente no es necesario si acabas de ejecutar la celda anterior)

if 'links_array' not in locals():
    print("Cargando 'junction_links.txt'...")
    links_array = np.loadtxt("junction_links.txt", skiprows=1)

if 'junction_coords' not in locals():
    print("Cargando 'junction_coordinates.txt'...")
    # Cargamos el archivo de coordenadas
    junction_coords_file_data = np.loadtxt("junction_coordinates.txt", skiprows=1)
    # Creamos el mapa de ID -> [x, y]
    j_coords_map = {int(row[0]): row[1:] for row in junction_coords_file_data}
else:
    # Si ya existe el array 'output_array' de la celda anterior
    j_coords_map = {int(row[0]): row[1:] for row in output_array}

# --- 2. Preparar datos para las flechas (Quiver) ---

# Obtenemos los midpoints EN ORDEN
# (Asumimos que el orden de df_tubos no ha cambiado desde que se creó links_array)
if len(df_tubos) != len(links_array):
    print("¡ADVERTENCIA! El número de tubos y de links no coincide.")
    print("Asegúrate de ejecutar la celda de exportación justo antes que esta.")

midpoints = np.stack(df_tubos['midpoint'].values)
U = np.zeros(len(links_array)) # Componente X del vector
V = np.zeros(len(links_array)) # Componente Y del vector

# Calculamos el vector de dirección para cada tubo
for i in range(len(links_array)):
    link = links_array[i]
    j_initial_id = int(link[0])
    j_final_id = int(link[1])
    
    # Obtenemos las coordenadas de las junctions
    coord_initial = j_coords_map[j_initial_id]
    coord_final = j_coords_map[j_final_id]
    
    # Calculamos el vector (final - inicial)
    vector = coord_final - coord_initial
    U[i] = vector[0]
    V[i] = vector[1]

# Normalizamos los vectores para que sean unitarios
norms = np.linalg.norm(np.vstack((U, V)), axis=0)
norms[norms == 0] = 1.0 # Evitar división por cero
U_norm = U / norms
V_norm = V / norms

# --- 3. Generar la visualización ---

fig, ax = plt.subplots(figsize=(14, 14), dpi=400)

# 3.1. Dibujar los granos (fondo)
for poly in final_circular_polygons:
    if poly.bounds[2] < roi_xmin or poly.bounds[0] > roi_xmax:
        continue
    x, y = poly.exterior.xy
    ax.fill(x, y, alpha=0.3, fc='gray', ec='none', label='Granos' if 'Granos' not in ax.get_legend_handles_labels()[1] else "")

# 3.2. Dibujar las flechas de flujo
ax.quiver(
    midpoints[:, 0], midpoints[:, 1], # Origen de la flecha (midpoint)
    U_norm, V_norm,                   # Dirección (vector unitario)
    color='black',
    scale=1200,                         # Escala (más grande = flechas más pequeñas)
    width=0.0003,                      # Ancho de la cabeza
    scale_units='xy',                 # Usar unidades de los ejes para escalar
    angles='xy',                      # No rotar flechas con el aspect ratio
    zorder=10
)

# 3.3. Configuración del gráfico
ax.set_aspect('equal')
ax.set_xlim(roi_xmin, roi_xmax)
ax.set_ylim(ymin, ymax) 
ax.set_title(f"Visualización de la Dirección del Flujo en Tubos")
ax.set_xlabel("Coordenada X (m)")
ax.set_ylabel("Coordenada Y (m)")
plt.grid(True, linestyle=':', alpha=0.6)
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

print("Iniciando visualización detallada de la red (Poiseuille)...")
print("(Este método es más robusto y puede tardar un poco más...)")

# --- 1. Preparación (Tangentes y Clasificación) ---
if 'N_SAMPLES' not in locals():
    N_SAMPLES = 10
    
# --- CAMBIO 1 ---
arrow_len = (roi_xmax - roi_xmin) * 0.002 # Longitud de flecha

oriented_tangents = {} 
Xg, Yg, Ug, Vg = [], [], [], [] 
junc1, junc2 = [], [] 
Xy, Yy, Uy, Vy = [], [], [], [] 

if 'tube_flow_map' not in locals():
    tube_flow_map = {idx: row['flow_rate'] for idx, row in df_tubos.iterrows()}
if 'junction_coords' not in locals():
    junction_coords = np.array([j['coord'] for j in junctions_list_structured])

print("Pre-calculando orientación de tangentes y clasificación...")
for tube_id, tube_data in df_tubos.iterrows():
    
    # --- 1a. Recalcular geometría ---
    idx1, idx2 = tube_data['bead_indices']
    is_wall_tube = tube_data['is_wall_tube']
    width = tube_data['width']
    
    if is_wall_tube:
        real_idx = idx1 if idx1 < n_real_beads else idx2
        c_real = centers[real_idx]; r_real = radii[real_idx]
        is_top_wall = c_real[1] > (ymax + ymin) / 2.0
        if is_top_wall:
            p_edge_bead = np.array([c_real[0], c_real[1] + r_real])
            p_edge_wall = np.array([c_real[0], ymax])
            n_vec_path = np.array([0.0, 1.0])
        else:
            p_edge_bead = np.array([c_real[0], c_real[1] - r_real])
            p_edge_wall = np.array([c_real[0], ymin])
            n_vec_path = np.array([0.0, -1.0])
    else:
        c1, c2 = all_centers[idx1], all_centers[idx2]
        r1, r2 = all_radii[idx1], all_radii[idx2]
        center_vec = c2 - c1; dist = np.linalg.norm(center_vec)
        n_vec_path = center_vec / dist
        p_edge_wall = c1 + n_vec_path * r1
        p_edge_bead = c2 - n_vec_path * r2
        
    # --- 1b. Orientar Tangente ---
    n_vec_flow_initial = np.array([-n_vec_path[1], n_vec_path[0]])
    sample_pts_k = np.linspace(p_edge_wall, p_edge_bead, N_SAMPLES)
    
    dists, idxs = tree.query(sample_pts_k, k=1)
    v_sec = U_cells[idxs]
    v_sec = np.nan_to_num(v_sec, nan=0.0, posinf=0.0, neginf=0.0)
    
    proj_mean = np.dot(v_sec, n_vec_flow_initial).mean()
    sign = np.sign(proj_mean) if np.sign(proj_mean) != 0 else 1.0
    t_oriented = n_vec_flow_initial * sign
    oriented_tangents[tube_id] = t_oriented
    
    midpoint = tube_data['midpoint']
    Xg.append(midpoint[0]); Yg.append(midpoint[1])
    Ug.append(t_oriented[0] * arrow_len); Vg.append(t_oriented[1] * arrow_len)

# --- 1c. Clasificar Junctions ---
for j_index, tube_id_list in enumerate(junctions_list):
    if j_index in border_junction_indices or not tube_id_list:
        continue
    flows = [tube_flow_map[t_id] for t_id in tube_id_list if t_id in tube_flow_map]
    if not flows:
        continue
    i_max_id = tube_id_list[np.argmax(flows)]
    midpoint_max = df_tubos.loc[i_max_id]['midpoint']
    j_coord = junction_coords[j_index]
    tangent_max = oriented_tangents[i_max_id]
    vec_m_to_j = j_coord - midpoint_max
    dp = tangent_max.dot(vec_m_to_j)
    if dp > 0: junc2.append(j_index)
    else: junc1.append(j_index)
    Xy.append(j_coord[0]); Yy.append(j_coord[1])
    Uy.append(tangent_max[0] * arrow_len); Vy.append(tangent_max[1] * arrow_len)

print(f"Pre-cálculo completado. Clasificadas {len(junc1)} tipo '1' y {len(junc2)} tipo '2'.")

# --- 2. ¡A PLOTEAR! ---
print("Generando gráfico...")
# --- CAMBIO 3 ---
fig, ax = plt.subplots(figsize=(14, 14), dpi=1200) # (Resolución muy alta)

# 2.1. Fondo (Granos)
print("Dibujando granos (vectorial)...")
for poly in final_circular_polygons:
    x, y = poly.exterior.xy
    ax.fill(x, y, alpha=0.4, fc='gray', ec='none', zorder=0)

# 2.2. Perfiles de Poiseuille (Líneas y Flechas cian)
print("Dibujando perfiles de tubo (Poiseuille)...")

# Calcular escala de velocidad
sample_vel_mags = np.linalg.norm(U_cells, axis=1)
max_vel = np.percentile(sample_vel_mags, 99.5) 
if max_vel == 0: max_vel = 1.0
vel_scale = 5 * arrow_len / max_vel 

for tube_id, tube_data in df_tubos.iterrows():
    
    # Recalcular geometría
    idx1, idx2 = tube_data['bead_indices']
    is_wall_tube = tube_data['is_wall_tube']
    if is_wall_tube:
        real_idx = idx1 if idx1 < n_real_beads else idx2
        c_real = centers[real_idx]; r_real = radii[real_idx]
        is_top_wall = c_real[1] > (ymax + ymin) / 2.0
        if is_top_wall:
            p_edge_bead = np.array([c_real[0], c_real[1] + r_real])
            p_edge_wall = np.array([c_real[0], ymax])
        else:
            p_edge_bead = np.array([c_real[0], c_real[1] - r_real])
            p_edge_wall = np.array([c_real[0], ymin])
    else:
        c1, c2 = all_centers[idx1], all_centers[idx2]
        r1, r2 = all_radii[idx1], all_radii[idx2]
        center_vec = c2 - c1; dist = np.linalg.norm(center_vec)
        n_vec_path = center_vec / dist
        p_edge_wall = c1 + n_vec_path * r1
        p_edge_bead = c2 - n_vec_path * r2
        
    pts_k = np.linspace(p_edge_wall, p_edge_bead, N_SAMPLES)
    xs, ys = pts_k[:,0], pts_k[:,1]
    
    dists, idxs = tree.query(pts_k, k=1)
    U_k = U_cells[idxs]
    U_k = np.nan_to_num(U_k, nan=0.0, posinf=0.0, neginf=0.0)
    uscaled_k = U_k * vel_scale
    
    # Dibujar ESTE tubo
    ax.plot(xs, ys, color='black', linewidth=0.5, alpha=0.6, zorder=3)
    ax.quiver(xs, ys,
              uscaled_k[:,0], uscaled_k[:,1],
              angles='xy', scale_units='xy', scale=1,
              width=0.001,
              color='cyan', alpha=0.8, zorder=2)

# 2.3. Tangentes de tubos (Flechas verdes)
# --- CAMBIO 4 (parámetros) ---
ax.quiver(Xg, Yg, Ug, Vg, angles='xy', scale_units='xy', scale=1,
          width=0.0004, headwidth=1, headlength=1.2, headaxislength=1,
          color='green', alpha=0.8, zorder=3, label='Tangente del Tubo (orientada)')

# 2.4. Tubo principal en Junction (Flechas amarillas)
# --- CAMBIO 4 (parámetros) ---
ax.quiver(Xy, Yy, Uy, Vy, angles='xy', scale_units='xy', scale=1,
          width=0.0004, headwidth=1, headlength=1.2, headaxislength=1,
          color='yellow', alpha=0.9, zorder=4, label='Tangente Tubo Principal')

# 2.5. Etiquetas de tipo de Junction (Textos 1 y 2)
for j_index in junc1:
    xj, yj = junction_coords[j_index]
    ax.text(xj, yj, "1", color='red', fontsize=3, fontweight='bold', ha='center', va='center', zorder=5)
for j_index in junc2:
    xj, yj = junction_coords[j_index]
    ax.text(xj, yj, "2", color='blue', fontsize=3, fontweight='bold', ha='center', va='center', zorder=5)

# 2.6. Configuración final
ax.set_aspect('equal')
ax.set_xlim(xmin, xmax)
ax.set_ylim(ymin, ymax)
ax.set_xlabel('x (m)')
ax.set_ylabel('y (m)')
ax.set_title('Visualización Detallada del Flujo')
ax.legend(loc='upper right')
plt.tight_layout()
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial import cKDTree

print("Iniciando el cálculo del gradiente de presión...")

# --- 1. Preparar el muestreo de Presión (similar a Velocidad) ---
RHO_FLUID = 1000.0 # Densidad del agua (kg/m^3)
P_NAME_CELL = None

# Buscar el campo de presión (OpenFOAM usa 'p' o 'p_rgh')
common_names = ['p', 'pressure', 'Pressure', 'p_rgh']
for name in common_names:
    if name in vol_mesh.cell_data:
        P_NAME_CELL = name
        break

if P_NAME_CELL is None:
    print("ERROR FATAL: No se encontró el campo de presión ('p' o 'p_rgh') en vol_mesh.cell_data.")
    # Si no se encuentra, detenemos esta celda
    pressure_gradients_np = np.array([0]) # Array vacío para evitar que el plot falle
else:
    print(f"Usando el campo de presión de celda: '{P_NAME_CELL}'")
    
    # 1a. Extraer datos y construir el árbol
    # (Usamos los mismos centros de celda que para la velocidad)
    if 'cell_centers_3d' not in locals():
         cell_centers_3d = vol_mesh.cell_centers().points
         
    p_data_raw = vol_mesh.cell_data[P_NAME_CELL] # (M,)
    
    # 1b. Construir el árbol en 2D
    tree_p = cKDTree(cell_centers_3d[:, :2])
    print("Árbol KDTree para presión construido.")

    # --- 2. Computar la presión en CADA junction ---
    
    # 2a. Obtener coordenadas de todas las junctions
    junction_coords = np.array([j['coord'] for j in junctions_list_structured])
    
    # 2b. Consultar el árbol para encontrar la celda más cercana a cada junction
    dists, idxs = tree_p.query(junction_coords, k=1)
    
    # 2c. Obtener los valores de presión (p/rho) y corregirlos
    p_junction_raw = p_data_raw[idxs]
    p_junction_corrected = p_junction_raw * RHO_FLUID # Presión real en Pascales
    
    # 2d. Crear un mapa para búsqueda fácil: {junction_id: Presión}
    junction_pressure_map = {j_idx: p for j_idx, p in enumerate(p_junction_corrected)}
    print(f"Presión muestreada y corregida para {len(junction_pressure_map)} junctions.")

    # --- 3. Calcular el gradiente para CADA tubo ---
    pressure_gradients = []
    
    # Iteramos sobre el mapa de direcciones (contiene j_initial y j_final)
    for tube_id, (j_initial_id, j_final_id) in tube_direction_map.items():
        
        # 3.1. Obtener presiones de las junctions conectadas
        p_initial = junction_pressure_map.get(j_initial_id)
        p_final = junction_pressure_map.get(j_final_id)
        
        if p_initial is None or p_final is None:
            continue
            
        # 3.2. Calcular la longitud (L) del tubo (distancia entre junctions)
        coord_initial = junctions_list_structured[j_initial_id]['coord']
        coord_final = junctions_list_structured[j_final_id]['coord']
        length = np.linalg.norm(coord_final - coord_initial)
        
        if length < 1e-12: # Evitar división por cero
            continue
            
        # 3.3. Calcular el gradiente de presión
        # (Usamos el valor absoluto, ya que el signo depende de la dirección)
        delta_p = p_final - p_initial
        gradient = np.abs(delta_p) / length # |(P2 - P1) / L|
        
        pressure_gradients.append(gradient)

    # Convertir a array de NumPy para estadísticas
    pressure_gradients_np = np.array(pressure_gradients)


# --- 4. Calcular Estadísticas ---
if pressure_gradients_np.size > 0:
    mean_grad = np.mean(pressure_gradients_np)
    var_grad = np.var(pressure_gradients_np)
    std_dev_grad = np.std(pressure_gradients_np)

    print("\n--- Pressure Gradient Statistics ---")
    print(f"Mean:\t\t {mean_grad:.2f} Pa/m")
    print(f"Variance:\t {var_grad:.2f} (Pa/m)^2")
    print(f"Std. Deviation:\t {std_dev_grad:.2f} Pa/m")
else:
    print("No se calcularon gradientes de presión.")
    mean_grad = 0

# --- 5. Representar el Histograma (NORMALIZADO) ---
print("Generando histograma (PDF)...")

fig, ax = plt.subplots(figsize=(10, 6), dpi=100)

if pressure_gradients_np.size > 0:
    # Limitar el rango del histograma para excluir outliers extremos
    p_min = np.percentile(pressure_gradients_np, 0.5)
    p_max = np.percentile(pressure_gradients_np, 99.5)
    
    # Asegurarse de que el rango es válido
    if p_max <= p_min:
        p_min = np.min(pressure_gradients_np)
        p_max = np.max(pressure_gradients_np)
    
    bins = np.linspace(p_min, p_max, 50)
    
    # --- CAMBIO AQUÍ ---
    ax.hist(pressure_gradients_np, bins=bins, color='darkorange', alpha=0.75, edgecolor='black', density=True)
    
    # Línea de la media
    ax.axvline(mean_grad, color='red', linestyle='--', linewidth=2, 
                label=f'Mean: {mean_grad:.2f} Pa/m')
    
    ax.set_title('Pressure Gradient Distribution (PDF)', fontsize=16) # <-- TÍTULO CAMBIADO
    ax.set_xlabel('Pressure Gradient (Pa/m)', fontsize=12)
    ax.set_ylabel('Probability Density', fontsize=12) # <-- ETIQUETA CAMBIADA
    ax.legend()
    ax.grid(True, linestyle=':', alpha=0.6)
    # Formato de notación científica si los números son muy grandes
    ax.ticklabel_format(axis='x', style='sci', scilimits=(0,0))
else:
    ax.set_title('Pressure Gradient Distribution (No Data)', fontsize=16)
    ax.set_xlabel('Pressure Gradient (Pa/m)', fontsize=12)
    ax.set_ylabel('Probability Density', fontsize=12) # <-- ETIQUETA CAMBIADA

plt.tight_layout()
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

print("Calculando la permeabilidad para cada tubo...")

# --- 1. Definir Constantes ---
H_METERS = 0.001 # h = 1 mm
R_CONST = H_METERS / np.sqrt(12) # r = h / sqrt(12)

permeabilities = []

# --- 2. Iterar sobre todos los tubos ---
# Usamos df_tubos, que ya contiene la 'width' efectiva
for tube_id, tube_data in df_tubos.iterrows():
    
    # 2.1. Get effective half-width (a)
    width = tube_data['width']
    a = width / 2.0
    
    # 2.2. Skip tubos con anchura cero o negativa
    if a < 1e-12: # Usar un épsilon pequeño para comparación de flotantes
        continue
        
    # 2.3. Aplicar la fórmula proporcionada
    h = H_METERS
    r = R_CONST
    
    ratio_r_a = r / a
    tanh_term = np.tanh(a / r) # Esto es tanh(1 / ratio_r_a)
    
    # k = (h^3 * a * (1 - (r/a) * tanh(a/r))) / 9
    k_numerator = (h**3 * a * (1 - ratio_r_a * tanh_term))
    k = k_numerator / 9.0
    
    permeabilities.append(k)

# --- 3. Calcular Estadísticas ---
permeabilities_np = np.array(permeabilities)

if permeabilities_np.size > 0:
    mean_k = np.mean(permeabilities_np)
    var_k = np.var(permeabilities_np)
    std_dev_k = np.std(permeabilities_np)

    print("\n--- Permeability Statistics ---")
    # Usamos notación científica (:.2e) porque las unidades (m^4) son inusuales
    print(f"Mean:\t\t {mean_k:.2e}")
    print(f"Variance:\t {var_k:.2e}")
    print(f"Std. Deviation:\t {std_dev_k:.2e}")
else:
    print("No se calcularon permeabilidades.")
    mean_k = 0

# --- 4. Representar el Histograma (NORMALIZADO) ---
print("Generando histograma (PDF)...")

fig, ax = plt.subplots(figsize=(10, 6), dpi=100)

if permeabilities_np.size > 0:
    # Limitar el rango para mejor visualización
    p_min = np.percentile(permeabilities_np, 0.5)
    p_max = np.percentile(permeabilities_np, 99.5)
    
    if p_max <= p_min: # Fallback si los percentiles fallan
        p_min = np.min(permeabilities_np)
        p_max = np.max(permeabilities_np)
    
    bins = np.linspace(p_min, p_max, 20)
    
    # density=True crea el histograma normalizado (PDF)
    ax.hist(permeabilities_np, bins=bins, color='seagreen', alpha=0.75, edgecolor='black', density=True)
    
    # Línea de la media
    ax.axvline(mean_k, color='red', linestyle='--', linewidth=2, 
                label=f'Mean: {mean_k:.2e}')
    
    ax.set_title('Permeability Distribution (PDF)', fontsize=16)
    ax.set_xlabel('Permeability ($k$)', fontsize=12) # No especifico unidades dado lo inusual de la fórmula
    ax.set_ylabel('Probability Density', fontsize=12)
    ax.legend()
    ax.grid(True, linestyle=':', alpha=0.6)
    ax.ticklabel_format(axis='x', style='sci', scilimits=(0,0))
else:
    ax.set_title('Permeability Distribution (No Data)', fontsize=16)
    ax.set_xlabel('Permeability ($k$)', fontsize=12)
    ax.set_ylabel('Probability Density', fontsize=12)

plt.tight_layout()
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

print("Calculando caudales teóricos (Q = k*G/mu)...")

# --- 1. Definir Constantes ---
MU_WATER = 1.0e-3 # Viscosidad dinámica del agua (Pa·s) en SI
H_METERS = 0.001  # h = 1 mm
R_CONST = H_METERS / np.sqrt(12) # r = h / sqrt(12)

# Comprobar que los mapas de las celdas anteriores existen
if 'tube_direction_map' not in locals() or 'junction_pressure_map' not in locals() or 'tube_flow_map' not in locals():
    print("ERROR: Faltan datos de celdas anteriores.")
    print("Por favor, ejecuta las celdas de 'Pre-cálculo de Dirección', 'Gradiente de Presión' y 'Cálculo de Tubos' primero.")
    # Crear arrays vacíos para que el script no falle
    real_flows_np = np.array([])
    theoretical_flows_np = np.array([])
else:
    real_flows = []
    theoretical_flows = []

    # --- 2. Iterar sobre todos los tubos válidos (los que tienen dirección) ---
    for tube_id, (j_initial_id, j_final_id) in tube_direction_map.items():
        
        # --- 2a. Obtener Caudal Real (Q_real) ---
        flow_real = tube_flow_map.get(tube_id)
        if flow_real is None:
            continue

        # --- 2b. Calcular Permeabilidad (k) ---
        tube_data = df_tubos.loc[tube_id]
        width = tube_data['width']
        a = width / 2.0 # Semianchura (a)
        
        if a < 1e-12:
            continue # Omitir tubos sin anchura
            
        # Fórmula de permeabilidad
        ratio_r_a = R_CONST / a
        tanh_term = np.tanh(a / R_CONST)
        k_numerator = (H_METERS**3 * a * (1 - ratio_r_a * tanh_term))
        k = k_numerator / 6.0
        k = 2.0*a*a*a*H_METERS/3.0
        
        
        # --- 2c. Calcular Gradiente de Presión (G) ---
        p_initial = junction_pressure_map.get(j_initial_id)
        p_final = junction_pressure_map.get(j_final_id)
        
        if p_initial is None or p_final is None:
            continue
        
        coord_initial = junctions_list_structured[j_initial_id]['coord']
        coord_final = junctions_list_structured[j_final_id]['coord']
        length = np.linalg.norm(coord_final - coord_initial)
        
        if length < 1e-12:
            continue
            
        delta_p = p_final - p_initial
        G = np.abs(delta_p) / length # Módulo del gradiente
        
        # --- 2d. Calcular Caudal Teórico (Q_theory) ---
        Q_theory = (k * G) / MU_WATER
        
        # --- 2e. Guardar ambos resultados ---
        real_flows.append(flow_real)
        theoretical_flows.append(Q_theory)

    # Convertir listas a arrays de NumPy
    real_flows_np = np.array(real_flows)
    theoretical_flows_np = np.array(theoretical_flows)

# --- 3. Calcular Estadísticas ---
if real_flows_np.size > 0:
    print("\n--- Real Flow Statistics ---")
    print(f"Mean:\t\t {np.mean(real_flows_np):.2e} m^3/s")
    print(f"Variance:\t {np.var(real_flows_np):.2e} (m^3/s)^2")
    
    print("\n--- Theoretical Flow Statistics ---")
    print(f"Mean:\t\t {np.mean(theoretical_flows_np):.2e} m^3/s")
    print(f"Variance:\t {np.var(theoretical_flows_np):.2e} (m^3/s)^2")
else:
    print("No se generaron datos de caudal para comparar.")

# --- 4. Representar los Histogramas (NORMALIZADOS) ---
print("Generando histogramas (PDF)...")
fig, ax = plt.subplots(figsize=(10, 6), dpi=100)

if real_flows_np.size > 0:
    # Encontrar un rango común para ambos histogramas
    combined_data = np.hstack((real_flows_np, theoretical_flows_np))
    p_min = np.percentile(combined_data, 0.5)
    p_max = np.percentile(combined_data, 99.5)
    
    if p_max <= p_min: # Fallback
        p_min = np.min(combined_data)
        p_max = np.max(combined_data)
    
    # Asegurarse de que el rango no sea cero
    if p_max == p_min:
         p_max = p_min + 1e-9 # Añadir un delta pequeño

    bins = np.linspace(p_min, p_max, 20)
    
    # Histograma de Caudal Real
    ax.hist(real_flows_np*H_METERS*1000, bins=bins, color='royalblue', alpha=0.7, 
            edgecolor='black', density=True, label='Real Flow Rate (CFD)')
    
    # Histograma de Caudal Teórico
    ax.hist(theoretical_flows_np, bins=bins, color='red', alpha=0.7, 
            edgecolor='black', density=True, label='Theoretical Flow Rate (k*G/mu)')
    
    ax.set_title('Real vs. Theoretical Flow Rate Distribution (PDF)', fontsize=16)
    ax.set_xlabel('Flow Rate ($Q, m^3/s$)', fontsize=12)
    ax.set_ylabel('Probability Density', fontsize=12)
    ax.legend()
    ax.grid(True, linestyle=':', alpha=0.6)
    ax.ticklabel_format(axis='x', style='sci', scilimits=(0,0))
else:
    ax.set_title('Flow Rate Distribution (No Data)', fontsize=16)
    ax.set_xlabel('Flow Rate ($Q, m^3/s$)', fontsize=12)
    ax.set_ylabel('Probability Density', fontsize=12)

plt.tight_layout()
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

print("Iniciando análisis de sensibilidad (k-CanalPlano vs G-Uniones)...")

# --- 1. Definir Constantes y verificar datos ---
MU_WATER = 1.0e-3 # Viscosidad dinámica del agua (Pa·s)
H_METERS = 0.001  # h = 1 mm

# Verificar que tenemos los datos de las uniones
if 'junction_pressure_map' not in locals() or 'tube_direction_map' not in locals():
    print("ERROR: Faltan datos (junction_pressure_map o tube_direction_map).")
    print("Ejecuta primero la celda de 'Cálculo del gradiente de presión'.")
else:
    q_real_list = []
    k_list = []
    g_list = []

    print("Calculando Q_real, k_tubo, y G_uniones para cada tubo...")
    # --- 2. Bucle unificado para recolectar datos alineados ---
    for tube_id, (j_initial_id, j_final_id) in tube_direction_map.items():
        
        # --- 2a. Obtener Q_real ---
        flow_real = tube_flow_map.get(tube_id, 0.0)
        
        # --- 2b. Calcular Permeabilidad (k_tubo) ---
        tube_data = df_tubos.loc[tube_id]
        width_cross_section = tube_data['width'] # Anchura efectiva
        a = width_cross_section / 2.0 # Semianchura (a)
        
        if a < 1e-12:
            k = 0.0
        else:
            # Fórmula de Canal Plano (k = 2*a^3*h / 3)
            k = (2.0 * (a**3) * H_METERS) / 3.0
            
        # --- 2c. Calcular Gradiente de Unión (G_uniones) ---
        p_initial = junction_pressure_map.get(j_initial_id)
        p_final = junction_pressure_map.get(j_final_id)
        
        coord_initial = junctions_list_structured[j_initial_id]['coord']
        coord_final = junctions_list_structured[j_final_id]['coord']
        length = np.linalg.norm(coord_final - coord_initial) # L_unión-unión
        
        if length < 1e-12:
            G = 0.0
        else:
            delta_p_uniones = p_final - p_initial
            G = np.abs(delta_p_uniones) / length # |(P_unión_B - P_unión_A)| / L_unión-unión
        
        # --- 2d. Guardar datos ---
        q_real_list.append(flow_real)
        k_list.append(k)
        g_list.append(G)

    # --- 3. Convertir a arrays y calcular promedios ---
    q_real_np = np.array(q_real_list)
    k_np = np.array(k_list)
    g_np = np.array(g_list)
    
    if q_real_np.size > 0:
        k_avg = np.mean(k_np)
        g_avg = np.mean(g_np)
        
        print(f"Valor promedio de Permeabilidad (k_avg_CanalPlano): {k_avg:.2e}")
        print(f"Valor promedio de Gradiente (G_avg_Uniones): {g_avg:.2e} Pa/m")

        # --- 4. Calcular las dos distribuciones teóricas ---
        q_theory_avg_k = (k_avg * g_np) / MU_WATER
        q_theory_avg_g = (k_np * g_avg) / MU_WATER

        # --- 5. Representar los tres histogramas (con líneas) ---
        print("Generando gráficos (PDF)...")
        fig, ax = plt.subplots(figsize=(10, 6), dpi=100)
        
        # Rango común
        combined_data = np.hstack((q_real_np, q_theory_avg_k, q_theory_avg_g))
        p_min = np.percentile(combined_data, 0.5)
        p_max = np.percentile(combined_data, 99.5)
        if p_max <= p_min: p_max = p_min + 1e-9
        
        bins = np.linspace(p_min, p_max, 50)
        
        # Calcular los datos del histograma (PDF)
        pdf_real, bin_edges = np.histogram(q_real_np, bins=bins, density=True)
        pdf_avg_k, _ = np.histogram(q_theory_avg_k, bins=bins, density=True)
        pdf_avg_g, _ = np.histogram(q_theory_avg_g, bins=bins, density=True)
        
        # Calcular los centros de los bins para el eje X
        bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2
        
        # --- INICIO DE LA MODIFICACIÓN (COLORES) ---
        
        # Graficar con ax.plot en lugar de ax.hist
        # Real Q (azul, 'C0', igual que line_h.get_color())
        ax.plot(bin_centers, pdf_real, 'o-', color='C0', label='Real Q (CFD)', markersize=5) 
        
        # Avg. k (rojo, 'C3', igual que pdf_mod)
        ax.plot(bin_centers, pdf_avg_k, '--', color='C3', label='Q (avg. k, real press. gradient)')
        
        # Avg. G (naranja, 'C1', igual que pdf_f)
        ax.plot(bin_centers, pdf_avg_g, '--', color='C1', label='Q (real k, avg. press. gradient)')
        
        # --- FIN DE LA MODIFICACIÓN (COLORES) ---
        
        #ax.set_title('Flow Rate Distribution Sensitivity (PDF)', fontsize=16)
        ax.set_xlabel('Flow Rate ($Q, m^3/s$)', fontsize=12)
        ax.set_ylabel('Probability Density', fontsize=12)
        ax.legend()
        ax.grid(True, linestyle=':', alpha=0.6)
        ax.ticklabel_format(axis='x', style='sci', scilimits=(0,0))
        
        plt.tight_layout()
        plt.show()

    else:
        print("No se generaron datos para el análisis.")